## Import stuff

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
import math
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())



2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14468512894105293593
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


# last value known for each patient + ratio between first and last

In [34]:
df = pd.read_csv("train_features.csv")
# df = pd.read_csv("test_features.csv")
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])

#RATIO
ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1)
ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
ratios = ratios.drop(["Time", "Age"], axis=1)
ratios.columns = [x+"_ratio" for x in ratios.columns]
ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)

# #MIN
# min = pd.DataFrame(df.groupby(["pid"]).min())
# min = pd.merge(orig_ordering["pid"], min, on="pid")
# min = min.drop(["Time", "Age"], axis=1)
# min.columns = [x+"_min" for x in min.columns]
# min.rename(columns={'pid_min': 'pid'}, inplace=True)

# #MAX
# max = pd.DataFrame(df.groupby(["pid"]).min())
# max = pd.merge(orig_ordering["pid"], max, on="pid")
# max = max.drop(["Time", "Age"], axis=1)
# max.columns = [x+"_max" for x in max.columns]
# max.rename(columns={'pid_max': 'pid'}, inplace=True)

# #MEAN
# mean = pd.DataFrame(df.groupby(["pid"]).mean())
# mean = pd.merge(orig_ordering["pid"], mean, on="pid")
# mean = mean.drop(["Time", "Age"], axis=1)
# mean.columns = [x+"_mean" for x in mean.columns]
# mean.rename(columns={'pid_mean': 'pid'}, inplace=True)

#LAST HOUR
for i in range(0, len(df), 12):
    patient = df.iloc[i:i+12]
    #fill each column with the last value
    for col in patient.columns:
        mean = patient[col].mean()
        if not math.isnan(mean):
            patient[col] = patient[col].dropna().iloc[-1]

    # print(patient)
    # break

last_hour = pd.DataFrame(np.array(df.iloc[:, 1:].values,  dtype=float)[11::12], columns=df.columns[1:])

#SQUARE
# square = last_hour**2

# merge the dataframes
df = pd.concat([last_hour, ratios], axis=1)
# df = pd.concat([last_hour, ratios, min, max, mean], axis=1)

#normalization
df2 = df.iloc[:,1:]
df2 = df2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df.iloc[:,1:] = df2

#make "pid" the first column
first_column = df.pop('pid')
df.insert(0, 'pid', first_column)
df.replace([np.inf, -np.inf], 0, inplace=True)
df.to_csv('train_features_ratio.csv', index=False, float_format='%.3f',na_rep='nan')
# df.to_csv('test_features_ratio.csv', index=False, float_format='%.3f',na_rep='nan')

<ipython-input-34-f9da4402671d>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].dropna().iloc[-1]


# imputation

In [13]:
df = pd.read_csv("train_features_ratio.csv")
# df = pd.read_csv("test_features_ratio.csv")
X = np.array(df.iloc[:, 1:].values,  dtype=float)

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(X)
df_imputed = pd.DataFrame(df_imputed)
df_imputed.insert(0, 'pid', first_column)
df_imputed.columns = df.columns
# df_imputed
df_imputed.to_csv('train_features_imputed.csv', index=False, float_format='%.3f',na_rep='nan')
# df_imputed.to_csv('test_features_imputed.csv', index=False, float_format='%.3f',na_rep='nan')











# EVERYTHING BELOW IS EXPERIMENTAL

In [37]:
last_hour**2

,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,196.0,1156.0,NaN,NaN,144.0,NaN,1444.0,72.25,676.0,0.0,...,NaN,10000.0,NaN,12321.0,492.84,4761.0,NaN,NaN,10404.0,54.6121
1,144.0,5041.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9604.0,NaN,NaN,NaN,5625.0,NaN,NaN,18496.0,NaN
2,169.0,4624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9409.0,NaN,NaN,NaN,12769.0,NaN,NaN,13456.0,NaN
3,144.0,6241.0,841.0,NaN,NaN,NaN,1444.0,NaN,NaN,NaN,...,NaN,9025.0,NaN,NaN,NaN,6241.0,NaN,NaN,15625.0,NaN
4,144.0,5776.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9801.0,NaN,NaN,NaN,8100.0,NaN,NaN,17424.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,144.0,6400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10000.0,NaN,NaN,NaN,12544.0,NaN,NaN,10201.0,NaN
18991,144.0,5329.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9216.0,NaN,NaN,NaN,4096.0,NaN,NaN,30276.0,NaN
18992,324.0,2809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10000.0,NaN,NaN,NaN,9025.0,NaN,NaN,9025.0,NaN
18993,144.0,7921.0,NaN,NaN,NaN,NaN,1369.0,NaN,NaN,NaN,...,NaN,10000.0,NaN,NaN,NaN,10000.0,NaN,NaN,24336.0,NaN


In [29]:
df = pd.read_csv("train_features.csv")
min = pd.DataFrame(df.groupby(["pid"]).min())
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
min = pd.merge(orig_ordering["pid"], min, on="pid")
min = min.drop(["Time", "Age"], axis=1)
min.columns = [x+"_min" for x in min.columns]
min.rename(columns={'pid_min': 'pid'}, inplace=True)
min

,pid,EtCO2_min,PTT_min,BUN_min,Lactate_min,Temp_min,Hgb_min,HCO3_min,BaseExcess_min,RRate_min,...,Alkalinephos_min,SpO2_min,Bilirubin_direct_min,Chloride_min,Hct_min,Heartrate_min,Bilirubin_total_min,TroponinI_min,ABPs_min,pH_min
0,1.0,NaN,NaN,12.0,NaN,36.0,8.5,24.0,-2.0,12.0,...,NaN,100.0,NaN,111.0,22.2,58.0,NaN,NaN,100.0,7.33
1,10.0,NaN,27.8,12.0,NaN,36.0,14.6,NaN,NaN,15.0,...,68.0,96.0,NaN,NaN,42.1,69.0,1.3,0.01,113.0,NaN
2,100.0,NaN,20.9,21.0,NaN,35.0,12.5,27.0,NaN,13.0,...,NaN,92.0,NaN,101.0,36.8,91.0,NaN,NaN,97.0,NaN
3,1000.0,26.0,NaN,22.0,3.66,36.0,9.2,NaN,NaN,12.0,...,NaN,95.0,NaN,NaN,27.3,69.0,NaN,NaN,118.0,7.30
4,10000.0,NaN,25.7,22.0,NaN,36.0,10.4,25.0,0.0,8.0,...,NaN,97.0,NaN,103.0,29.3,64.0,NaN,NaN,95.0,7.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,NaN,NaN,13.0,NaN,35.0,7.4,NaN,NaN,16.0,...,64.0,95.0,NaN,NaN,24.0,102.0,0.5,9.48,90.0,NaN
18991,9995.0,NaN,55.5,50.0,NaN,36.0,11.2,29.0,NaN,15.0,...,NaN,90.0,NaN,89.0,35.6,58.0,NaN,NaN,162.0,NaN
18992,9996.0,NaN,NaN,NaN,NaN,37.0,10.7,NaN,-1.0,10.0,...,NaN,96.0,NaN,NaN,25.1,86.0,NaN,NaN,80.0,7.36
18993,9998.0,NaN,34.7,13.0,NaN,36.0,8.4,NaN,NaN,18.0,...,NaN,100.0,NaN,NaN,25.6,90.0,NaN,0.85,109.0,NaN


In [64]:
df = pd.read_csv("train_features.csv")
ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1)
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
ratios = ratios.drop(["Time", "Age"], axis=1)
ratios.columns = [x+"_ratio" for x in ratios.columns]
ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)
ratios

,pid,EtCO2_ratio,PTT_ratio,BUN_ratio,Lactate_ratio,Temp_ratio,Hgb_ratio,HCO3_ratio,BaseExcess_ratio,RRate_ratio,...,Alkalinephos_ratio,SpO2_ratio,Bilirubin_direct_ratio,Chloride_ratio,Hct_ratio,Heartrate_ratio,Bilirubin_total_ratio,TroponinI_ratio,ABPs_ratio,pH_ratio
0,1.0,NaN,NaN,0.000000,NaN,0.055556,-0.022989,0.083333,-1.0,0.125000,...,NaN,0.000000,NaN,-0.026316,-0.097561,-0.265957,NaN,NaN,-0.281690,0.008186
1,10.0,NaN,0.00000,0.000000,NaN,0.000000,0.000000,NaN,NaN,-0.304348,...,0.000000,-0.010101,NaN,NaN,0.000000,-0.085366,0.000000,0.000000,-0.062069,NaN
2,100.0,NaN,0.00000,0.000000,NaN,0.085714,0.000000,0.000000,NaN,-0.071429,...,NaN,-0.030000,NaN,0.000000,0.000000,0.202128,NaN,NaN,-0.265823,NaN
3,1000.0,0.115385,NaN,0.000000,-0.096296,0.055556,0.000000,NaN,NaN,0.000000,...,NaN,-0.050000,NaN,NaN,0.000000,-0.281818,NaN,NaN,0.008065,0.000000
4,10000.0,NaN,0.22179,0.000000,NaN,0.027778,-0.054545,0.040000,inf,-0.272727,...,NaN,-0.010000,NaN,-0.009615,-0.063898,0.406250,NaN,NaN,0.200000,0.006803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,NaN,NaN,0.076923,NaN,-0.027778,0.256757,NaN,NaN,-0.280000,...,-0.030303,0.052632,NaN,NaN,0.179167,-0.164179,-0.166667,0.253165,-0.151261,NaN
18991,9995.0,NaN,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,0.235294,...,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,-0.005714,NaN
18992,9996.0,NaN,NaN,NaN,NaN,0.027027,-0.018349,NaN,NaN,0.800000,...,NaN,0.000000,NaN,NaN,-0.239394,0.104651,NaN,NaN,-0.159292,0.002699
18993,9998.0,NaN,0.00000,0.000000,NaN,0.027778,0.000000,NaN,NaN,0.052632,...,NaN,0.000000,NaN,NaN,0.000000,-0.056604,NaN,-0.606481,0.431193,NaN


# very old stuff

In [14]:
df = pd.read_csv("train_features.csv")
# patients = pd.read_csv("test_features.csv")

# go through 12 rows at a time
for i in range(0, len(df), 12):
    # get the 12 rows
    patient = df.iloc[i:i+12]
    #fill each column with the last value
    for col in patient.columns:
        mean = patient[col].mean()
        if not math.isnan(mean):
            # print(col)
            # print(patient[col].dropna().iloc[-1])
            patient[col] = patient[col].dropna().iloc[-1]

    # print(patient)
    break

last_hour = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])

#normalize each column except the first 2
last_hour2 = last_hour.iloc[:,2:]
last_hour2 = last_hour2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
last_hour.iloc[:,2:] = last_hour2
last_hour


<ipython-input-14-2400c3f435fd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].dropna().iloc[-1]


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1.0,14.0,0.223529,NaN,NaN,0.053398,NaN,0.666667,0.185792,0.475,...,NaN,1.000000,NaN,0.697674,0.153191,0.262821,NaN,NaN,0.331658,0.743243
1,10.0,12.0,0.658824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.959184,NaN,NaN,NaN,0.301282,NaN,NaN,0.502513,NaN
2,100.0,13.0,0.623529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.938776,NaN,NaN,NaN,0.544872,NaN,NaN,0.402010,NaN
3,1000.0,12.0,0.752941,0.275362,NaN,NaN,NaN,0.666667,NaN,NaN,...,NaN,0.897959,NaN,NaN,NaN,0.326923,NaN,NaN,0.447236,NaN
4,10000.0,12.0,0.717647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.979592,NaN,NaN,NaN,0.397436,NaN,NaN,0.482412,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,12.0,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.538462,NaN,NaN,0.326633,NaN
18991,9995.0,12.0,0.682353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.918367,NaN,NaN,NaN,0.230769,NaN,NaN,0.693467,NaN
18992,9996.0,18.0,0.447059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.429487,NaN,NaN,0.296482,NaN
18993,9998.0,12.0,0.870588,NaN,NaN,NaN,NaN,0.555556,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.461538,NaN,NaN,0.603015,NaN


## code to remove nan values (obsolete)

In [6]:
patients = pd.read_csv("train_features.csv")
# patients = pd.read_csv("test_features.csv")

patients.interpolate(method='linear', inplace=True)
#get the mean for each column
mean_values = patients.mean()


# go through 12 rows at a time
for i in range(0, len(patients), 12):
    # get the 12 rows
    patient = patients.iloc[i:i+12]
    #for each column, use linear interpolation to replace the nan values
    for col in patient.columns:
        patient[col] = patient[col].interpolate(method='linear')
        # mean = patient[col].mean()
        # if math.isnan(mean):
        #     patient[col] = mean_values[col]
        mean = patient[col].mean()
        patient[col] = patient[col].fillna(mean)



    if i%1000 == 0:
        print(i)
        # break
    # break



#save the dataframe
# patients.to_csv("train_features_fixed.csv", index=False, float_format='%.3f',na_rep='nan')
patients.to_csv("train_features_fixed_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')
# patients.to_csv("test_features_fixed_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')




<ipython-input-6-8cddb51b224d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].interpolate(method='linear')
<ipython-input-6-8cddb51b224d>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].fillna(mean)


0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
132000
135000
138000
141000
144000
147000
150000
153000
156000
159000
162000
165000
168000
171000
174000
177000
180000
183000
186000
189000
192000
195000
198000
201000
204000
207000
210000
213000
216000
219000
222000
225000


# (obsolete)

In [7]:
# patients = pd.read_csv("train_features_fixed.csv")
# patients = pd.read_csv("test_features_fixed.csv")
patients = pd.read_csv("train_features_fixed_withnancols.csv")

#normalize each column except the first 2
patients2 = patients.iloc[:,2:]
patients2 = patients2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
patients.iloc[:,2:] = patients2


#save the dataframe
patients.to_csv("train_features_normalized_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')

In [23]:
# s = pd.Series([np.nan, 2, 4, 8,np.nan,12])
# s = s.interpolate(method='quadratic')
# s

0          NaN
1     2.000000
2     4.000000
3     8.000000
4    10.842105
5    12.000000
dtype: float64